In [1]:
import pandas as pd
import os

# Пути к файлам
combined_scores_path = r"../../depmap_datasets/prepared_datasets/depscores.csv"
merged_data_path = r"../../depmap_datasets/prepared_datasets/cancer_cell_lines_merged_data.csv"

# Чтение файлов
df_combined = pd.read_csv(combined_scores_path)
df_merged = pd.read_csv(merged_data_path)

# Список заболеваний
diseases = ['AML', 'CML', 'CLL', 'ALL', 'Lung Cancer', 'Neuroblastoma', 'Colon/Colorectal Cancer', 'Ovarian Cancer']

# Заданные пороги для бинаризации депскоров
thresholds = [0.0, -0.05, -0.1, -0.15, -0.2, -0.25, -0.3, -0.35, -0.4, -0.45, -0.5, -0.55, -0.6, -0.65, -0.7, -0.75, -0.8, -0.85, -0.9, -0.95, -1.0, -1.05, -1.1, -1.15, -1.2, -1.25, -1.3, -1.35, -1.4, -1.45, -1.5]

# Для каждого типа рака
for disease in diseases:
    # Отбираем строки, соответствующие текущему заболеванию
    df_disease = df_merged[df_merged['primary_disease_renamed'] == disease]
    df_valid_lines = df_combined[df_combined['DepMap_ID'].isin(df_disease['ModelID'])]

    # Добавляем столбец с типом заболевания
    df_valid_lines = pd.merge(df_valid_lines, df_disease[['ModelID', 'primary_disease_renamed']], left_on='DepMap_ID', right_on='ModelID', how='left')

    # Убираем ненужные столбцы
    df_genes = df_valid_lines.drop(columns=['DepMap_ID', 'ModelID', 'primary_disease_renamed'])

    # Фильтруем только числовые столбцы
    df_genes_numeric = df_genes.select_dtypes(include=['number'])

    # Группировка по типу рака и вычисление среднего значения для каждого гена
    df_avg_scores = df_genes_numeric.mean()

    # Создаём новый датафрейм для бинарных значений
    binary_df = df_avg_scores.to_frame(name='Average_Score')

    # Применяем бинаризацию для каждого порога
    for threshold in thresholds:
        binary_df[f"<= {threshold}"] = df_avg_scores.apply(lambda x: "Да" if x <= threshold else "Нет")

    # Вычисляем проценты "Да" в каждом бинарном столбце
    percent_row = {"Average_Score": "Проценты"}
    for threshold in thresholds:
        percent_yes = (binary_df[f"<= {threshold}"] == "Да").mean() * 100
        percent_row[f"<= {threshold}"] = percent_yes  # Без округления

    # Добавляем строку "Проценты" в начало датафрейма
    binary_df = pd.concat([pd.DataFrame([percent_row]), binary_df])

    # Обрабатываем название для файла (замена "/" на "-")
    safe_disease_name = disease.replace("/", "-")

    # Определяем путь сохранения
    output_dir = r"./mean_depscores_by_cancer_type/"
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"mean_depscore_{safe_disease_name.replace('/', '_').lower()}.csv")

    # Сохраняем данные без индекса
    binary_df.to_csv(output_file)

    print(f"Файл с бинарными данными для {disease} сохранён по пути: {output_file}")


Файл с бинарными данными для AML сохранён по пути: ./mean_depscores_by_cancer_type/mean_depscore_aml.csv
Файл с бинарными данными для CML сохранён по пути: ./mean_depscores_by_cancer_type/mean_depscore_cml.csv
Файл с бинарными данными для CLL сохранён по пути: ./mean_depscores_by_cancer_type/mean_depscore_cll.csv
Файл с бинарными данными для ALL сохранён по пути: ./mean_depscores_by_cancer_type/mean_depscore_all.csv
Файл с бинарными данными для Lung Cancer сохранён по пути: ./mean_depscores_by_cancer_type/mean_depscore_lung cancer.csv
Файл с бинарными данными для Neuroblastoma сохранён по пути: ./mean_depscores_by_cancer_type/mean_depscore_neuroblastoma.csv
Файл с бинарными данными для Colon/Colorectal Cancer сохранён по пути: ./mean_depscores_by_cancer_type/mean_depscore_colon-colorectal cancer.csv
Файл с бинарными данными для Ovarian Cancer сохранён по пути: ./mean_depscores_by_cancer_type/mean_depscore_ovarian cancer.csv
